# Chapter 9: Classes (part 6)

## Class Attributes & Instance Attributes at Superclass

So far, when dealing with attributes, they have been defined at the class level, but they have a different value in each instance. We describe these as instance attributes.

What if we wanted an attribute to have a shared value between all instances of the class?

What if we wanted an attribute to be defined for all subclasses, but have a different value in each instance?

In [ ]:
class Shape:
    """Shape v2 (class attribute)"""

    __count = 0

    def __init__(self, name = None):
        self.__count += 1
        self.__set_name(name)

    def __del__(self):
        self.__count -= 1

    def calculate_area(self):
        return 0

    def __str__(self):
        return 'We should not get here, this is a Shape with area %.2f' % self.calculate_area()

    def __gt__(self, other):
        return self.calculate_area() > other.calculate_area()

    def __set_name(self, name):
        self.__name = name

    def __get_name(self):
        return self.__name

    name = property(__get_name, __set_name)

    def __get_count(self):
        return self.__count

    count = property(__get_count)

# or
# from Shape_2 import Shape

`count` only has a getter: this is how to define a read-only property. If you try assigning to it, it will fail.

In [ ]:
sh = Shape('sh')
sh.count

In [ ]:
sh.count = 42

In [ ]:
import math

class Circle(Shape):
    """Circle v8 (class attribute)"""
    def __init__(self, radius = 0.0, name = ''):
        super().__init__(name)
        self.__set_radius(radius)

    def calculate_area(self):
        return math.pi * (self.__radius ** 2)

    def __set_radius(self, radius):
        if radius >= 0.0:
            self.__radius = radius
        else:
            print('radius cannot be less than 0.0')
            self.__radius = 0.0

    def __get_radius(self):
        return self.__radius

    def __str__(self):
        return 'Circle with radius %.2f' % self.__get_radius()

    radius = property(__get_radius, __set_radius)

# or
# from Circle_8 import Circle

In [ ]:
c8a = Circle(1.0, 'c8a')
c8b = Circle(1.5, 'c8b')

In [ ]:
c8a.__dict__

In [ ]:
c8b.__dict__

We can see that, as expected, the two instances of `Circle` have different names. Notice that the property is specially _mangled_ to show that it originates from `Shape`.

However, the counts are not what we expected. The property was defined in `Shape` and we expected it to count instances of that class or subclasses.

The problem is that each instance operates as a separate namespace, referred to by `self` and we manipulated the count as `self.__count`. We have created an instance attribute at the superclass level. This is often useful, but it is not what we were looking for here.

We need a different approach.

In [ ]:
class Shape:
    """Shape v3 (class attribute fixed)"""

    __count = 0

    def __init__(self, name = ''):
        Shape.__count += 1
        self.__set_name(name)

    def __del__(self):
        Shape.__count -= 1

    def calculate_area(self):
        return 0

    def __str__(self):
        return 'We should not get here, this is a Shape with area %.2f' % self.calculate_area()

    def __gt__(self, other):
        return self.calculate_area() > other.calculate_area()

    def __set_name(self, name):
        self.__name = name

    def __get_name(self):
        return self.__name

    name = property(__get_name, __set_name)

    def __get_count(self):
        return Shape.__count

    count = property(__get_count)

# or
# from Shape_3 import Shape

In [ ]:
import math

class Circle(Shape):
    """Circle v8 (class attribute)"""
    def __init__(self, radius = 0.0, name = ''):
        super().__init__(name)
        self.__set_radius(radius)

    def calculate_area(self):
        return math.pi * (self.__radius ** 2)

    def __set_radius(self, radius):
        if radius >= 0.0:
            self.__radius = radius
        else:
            print('radius cannot be less than 0.0')
            self.__radius = 0.0

    def __get_radius(self):
        return self.__radius

    def __str__(self):
        return 'Circle with radius %.2f' % self.__get_radius()

    radius = property(__get_radius, __set_radius)

# or
# from Circle_8b import Circle

In [ ]:
c8a = Circle(1.0)
c8b = Circle(1.5)

In [ ]:
c8a.__dict__

In [ ]:
print(c8a.count)
print(c8b.count)

In [ ]:
del c8b
print(c8a.count)
print(c8b.count)

In [ ]:
Shape.count

It works now, but we can only access it through an instance rather than through the class. There are ways around this, but none of them really replicate static members in languages like Java or C++.

Further details are beyond the scope of this course.

## Exercise 9.5

Now re-open Exercise 9 and complete the section for Exercise 9.5.

## Common Patterns in Inheritance

There is one very common pattern in inheritance. That is to have the class `__init__()` functions defined to accept keyword parameters. The parameters that will be bound in that class are named keywords, and the rest are covered by a `**kwds` parameter to be passed on to the parent.

If we apply this pattern to our classes, we might end up with this:

In [ ]:
import math

class Circle(Shape):
    """Circle v9 (kwds init)"""
    def __init__(self, radius = 0.0, **kwds):
        super().__init__(**kwds)
        self.__set_radius(radius)

    def calculate_area(self):
        return math.pi * (self.__radius ** 2)

    def __set_radius(self, radius):
        if radius >= 0.0:
            self.__radius = radius
        else:
            print('radius cannot be less than 0.0')
            self.__radius = 0.0

    def __get_radius(self):
        return self.__radius

    def __str__(self):
        return 'Circle with radius %.2f' % self.__get_radius()

    radius = property(__get_radius, __set_radius)

# or
# from Circle_9 import Circle

- The advantage here is that the subclass does not need to handle each superclass attribute indvidually

In [ ]:
c9a = Circle(2.0)
print(c9a.name, end='**\n')
print(c9a)

In [ ]:
c9b = Circle(1.0, name='Secundus')
print(c9b.name, end='**\n')
print(c9b)

In [ ]:
c9b = Circle(1.0, 'Secundus')
print(c9b.name, end='**\n')
print(c9b)

- But the trade-off is that we can no longer call the constructor without using named parameters.

## Multiple Inheritance

Python supports multiple inheritance using the following syntax:
```
class SubClassName(Parent1, Parent2):
```
The MRO checks `Parent1` first and then `Parent2`.

This is beyond the scope of this course.

# End of Notebook